In [97]:
import math
import pygame
import tkinter as tk
from tkinter import *

In [98]:
# Bodies are considered to be circular and 2-dimensional
# They are placed in relation to an origin (0, 0) on a 2d coordinate grid

class Body:
    def __init__(self, mass, radius, x_coordinate, y_coordinate, name, is_immobile, x_velocity=0, y_velocity=0, x_acceleration=0, y_acceleration=0):
        # Standard physics symbols will be used, e.g. the lowercase letter m for mass
        # Standard physics units will also be used, e.g. the unit kg for mass
        self.m = mass
        self.r = radius
        self.name = name
        self.is_immobile = is_immobile
        self.vx, self.vy = x_velocity, y_velocity
        self.v = math.sqrt(self.vx**2+self.vy**2)
        self.ax, self.ay = x_acceleration, y_acceleration
        self.a = math.sqrt(self.ax**2+self.ay**2)
        self.x, self.y = x_coordinate, y_coordinate
        
        
    # The body is moved along its velocity vectors for t seconds.
    def move(self, t):
        if self.is_immobile:
            self.vx, self.vy, self.v, self.a, self.ax, self.ay = 0, 0, 0, 0, 0, 0
        self.x += self.vx*t
        self.y += self.vy*t

In [99]:
class GravitySimulation:
    # Time step is how many seconds each body is moved along its velocity vector each simulation step
    def __init__(self, time_step):
        self.bodies = [] # A simple list to keep track of the bodies objects in the simulation
        self.num_bodies = 0 # A helper variable to avoid constant len() calls
        self.t = time_step
        self.G = 6.6743e-11 # The gravitational constant, used to calculate gravitational force between bodies
                
        
    def add_body(self, mass, radius, x_coordinate, y_coordinate, name, is_immobile, x_velocity=0, y_velocity=0):
        new_body = Body(mass, radius, x_coordinate, y_coordinate, name, is_immobile, x_velocity, y_velocity)
        self.num_bodies += 1
        self.bodies.append(new_body)
        
        
    # Calculates the sum of all gravitational forces between all pairs of objects and adjusts their velocity vectors accordingly
    # Then the objects are moved along their velocity vectors for t time.
    def step(self):
        # Gravity is calculated between pairs of bodies, so all pairs of bodies must be found.
        # This code puts all pairs of objects in tuples in the pairs list
        merge_occured = False # If two bodies were merged in this step operation
        pairs = []
        for i in range(self.num_bodies):
            for j in range(i+1, self.num_bodies):
                pairs.append((self.bodies[i], self.bodies[j])) 
                self.bodies[i].ax = 0
                self.bodies[i].ay = 0 
                self.bodies[j].ax = 0
                self.bodies[j].ay = 0             
                
        for i in pairs:
            b1, b2 = i[0], i[1] # Stands for body 1 and body 2. Short variable names are used for shorter code.
            d = math.sqrt((b1.x-b2.x)**2+(b1.y-b2.y)**2) # d is the distance between the centers of the two bodies.
            if d < b1.r+b2.r: # Two bodies have collided. They are now merged.
                self.add_body(b1.m+b2.m, math.sqrt(b1.r**2+b2.r**2), (b1.x+b2.x)/2, (b1.y+b2.y)/2, f"Merged body {self.num_bodies-1}", False)
                self.bodies[-1].vx = b1.vx + b2.vx
                self.bodies[-1].vy = b1.vy + b2.vy
                self.bodies = [b for b in self.bodies if b.name not in (b1.name, b2.name)]
                self.num_bodies -= 2
                merge_occured = True                
            else: # Update the acceleration vectors of each body due to gravity
                # The magnitude of the acceleration due to gravity on each body
                a1 = (self.G*b2.m)/d**2
                a2 = (self.G*b1.m)/d**2
                # The angle in radians of the acceleration vector with respect to the x axis for each body (e.g. 90 degrees is North)
                # math.atan2 is used instead of math.atan as it considers the sign of the b2.y-b1.y and b2.x-b1.x terms
                θ1 = math.atan2(b2.y-b1.y, b2.x-b1.x)
                θ2 = math.atan2(b1.y-b2.y, b1.x-b2.x)
                # The acceleration vectors due to gravity on each body in the x and y axes
                da1x, da1y = a1*math.cos(θ1), a1*math.sin(θ1)
                da2x, da2y = a2*math.cos(θ2), a2*math.sin(θ2)
     
                # Add these acceleration vectors to each body in their respective direction
                b1.ax += da1x
                b1.ay += da1y
                b2.ax += da2x
                b2.ay += da2y
 
        # Finally, update each body's velocity vectors and move each body along its velocity vectors for t seconds
        for b in self.bodies:
            b.vx += b.ax*self.t
            b.vy += b.ay*self.t
            b.v = math.sqrt(b.vx**2+b.vy**2)
            b.a = math.sqrt(b.ax**2+b.ay**2)
            b.move(self.t)
        return merge_occured
            
            
    # For this to be guaranteed to work:
        # The body to orbit around must be immobile
        # There are no other bodies in the simulation
    # Changing the semi-major axis length can form elliptical orbits.
    def generate_stable_orbit(self, orbiting_body, body_to_orbit_around, semimajor_axis_length):
        b1, b2 = orbiting_body, body_to_orbit_around # Assign shorter variable names for shorter codes
        l = semimajor_axis_length # l stands for 'length', here the length of the semi-major axis
        d = math.dist((b1.x, b1.y), (b2.x, b2.y)) # d is the distance between the centers of the two bodies.
        if l < d/2: # The length of the semi-major axis is impossibly short
            raise ValueError("The Semimajor axis length cannot be less than half the distance between the bodies")
        vo = math.sqrt(self.G*b2.m*(2/d-1/l)) # The magnitude of the orbital velocity vector for the body that is orbiting
        θ = math.atan2(b2.y-b1.y, b2.x-b1.x) # The angle from the orbiting body to the body it is orbiting aroudn
        vox, voy = vo*math.cos(-math.pi/2-θ), -vo*math.sin(-math.pi/2-θ) # Extract the velocities in the x and y directions
        b1.vx, b1.vy = vox, voy # Set the orbiting body's velocities to the orbit velocities in their respective direction
        b1.v = math.sqrt(b1.vx**2+b1.vy**2)
        

In [100]:
class GUI:
    def __init__(self):
        pygame.init()
        self.fps = 60 # Stands for frames per second, equivalent to simulation steps per second
        self.sim = GravitySimulation(1) # Default time step of 1 second, changeable by the user
        self.window = pygame.display.set_mode((0, 0), pygame.FULLSCREEN)
        self.root = tk.Tk() # The backend tkinter.
        self.width, self.height = self.window.get_size() # Width and height of the window in pixels
        self.body_scaling = 1 # A simple scale factor for the size of the bodies, changeable by zooming in or out
        self.square_length = 30 # The length of the small grid squares on the screen
        self.x_offset, self.y_offset = 0, 0 # Offset of grid and bodies to allow for screen dragging
        self.velocity_arrow_scale = 1
        self.acceleration_arrow_scale = 1
        self.clock = pygame.Clock()
        self.mouse_held = False
        self.paused = True
        self.adding_body = False
        self.selected_body = None
        self.centered_body = None
        self.init_tkinter()
                    
        # ----------------------------------------
        # Add initial bodies
              
        # Earth and moon to scale  
        self.sim.add_body(5.97219*10**24, 6378.1*10**3, 0, 0, "Earth", True)
        self.sim.add_body(7.35*10**22, 1737.4*10**3, 384400*10**3, 0, "Moon", False)
        self.sim.generate_stable_orbit(self.sim.bodies[1], self.sim.bodies[0], 384400*10**3)
        
        # Binary system
        # self.sim.add_body(10**14, 10, 150, 0, "Body 1", False, y_velocity=2)
        # self.sim.add_body(10**14, 10, -150, 0, "Body 2", False, y_velocity=-2)
        
        self.sim.step()
        self.gui_mainloop()
        
        
    def init_tkinter(self):
        self.root.configure(background="black", highlightcolor="white", highlightthickness=1)
        self.root.geometry(f"275x{self.height}+0+0")
        self.root.resizable(False, False)
        self.root.overrideredirect(True)
        self.root.attributes('-topmost', True)
        
        Button(self.root, text="Quit simulation", command=self.kill).grid()
        Button(self.root, text="Pause simulation", command=self.un_pause_or_pause).grid()
        Button(self.root, text="Step simulation", command=self.manual_step).grid()
        
        Label(self.root, text ="FPS Slider").grid()
        fps_slider = Scale(self.root, command=self.set_fps, orient=HORIZONTAL, from_=1, to=1000)
        fps_slider.set(60)
        fps_slider.grid()
        
        Label(self.root, text ="Timestep slider").grid()
        time_step_slider = Scale(self.root, command=self.set_timestep, orient=HORIZONTAL, from_=0.01, to=100, resolution=0.01)
        time_step_slider.set(self.sim.t)
        time_step_slider.grid()
        
        Label(self.root, text="Velocity arrow scaling").grid()
        velocity_arrow_scale_slider = Scale(self.root, command=self.set_velocity_arrow_scale, orient=HORIZONTAL, from_=0.01, to=100, resolution=0.01)
        velocity_arrow_scale_slider.set(self.velocity_arrow_scale)
        velocity_arrow_scale_slider.grid()
        
        Label(self.root, text="Acceleration arrow scaling").grid()
        acceleration_arrow_scale_slider = Scale(self.root, command=self.set_acceleration_arrow_scale, orient=HORIZONTAL, from_=0.01, to=100, resolution=0.01)
        acceleration_arrow_scale_slider.set(self.acceleration_arrow_scale)
        acceleration_arrow_scale_slider.grid()
                
        body_info_frame = Frame(self.root, background="white")
        body_info_frame.grid()   
        
        self.selected_body_info = {"name": StringVar(self.root, "-"), "m": StringVar(self.root, "-"), "r": StringVar(self.root, "-"), 
                                   "x": StringVar(self.root, "-"), "y": StringVar(self.root, "-"), "v": StringVar(self.root, "-"), 
                                   "vx": StringVar(self.root, "-"), "vy": StringVar(self.root, "-"), 
                                   "is_immobile": StringVar(self.root, "-"), "a": StringVar(self.root, "-"),
                                   "ax": StringVar(self.root, "-"), "ay": StringVar(self.root, "-"),}
        
        
        Label(body_info_frame, text="Selected body information").grid()
        
        self.body_info_entries = {}
        
        for i,v in enumerate((("name", "Name: "), ("m", "Mass (kg): "), ("r", "Radius (m)"), 
                              ("x", "x-coordinate: "), ("y", "y-coordinate: "), 
                              ("is_immobile", "is immobile?: "), ("v", "Velocity (m s⁻¹): "), 
                              ("vx", "x velocity (m s⁻¹): "), ("vy", "y velocity (m s⁻¹): "), ("a", "acceleration (m s⁻²)"),
                              ("ax", "x acceleration (m s⁻²)"), ("ay", "y acceleration (m s⁻²)"))):
            Label(body_info_frame, text=v[1]).grid(column=0)
            temp = Entry(body_info_frame, textvariable=self.selected_body_info[v[0]])
            temp.bind("<FocusIn>", lambda event:self.pause())
            temp.bind("<Return>", lambda event, field=v[0]:self.set_selected_body_field(field))
            temp.grid(row=i+1, column=1)
            self.body_info_entries[v[0]] = temp
            
        Button(body_info_frame, text="Delete selected body", command=lambda:self.delete_selected_body()).grid()
        Button(body_info_frame, text="Center on selected body", command=lambda:self.center_on_selected_body()).grid()
        Button(body_info_frame, text="De-center on selected body", command=lambda:self.decenter_on_selected_body()).grid()
            
        Label(self.root, background="black").grid()
                
        orbit_generator_frame = Frame(self.root, background="white")
        orbit_generator_frame.grid()
        Label(orbit_generator_frame, text="Stable orbit generator").grid()
        
        self.orbit_generator_inputs = {"orbiting_body": StringVar(self.root, "-"), 
                                       "body_to_orbit_around": StringVar(self.root, "-"), 
                                       "semimajor_axis_length": StringVar(self.root, "-")}
        self.orbit_generator_optionmenus = {}
        
        for i,v in enumerate((("orbiting_body", "Orbiting body: "), ("body_to_orbit_around", "Body to orbit around: "))):
            Label(orbit_generator_frame, text=v[1]).grid(column=0)
            temp = OptionMenu(orbit_generator_frame, self.orbit_generator_inputs[v[0]], [])
            temp.grid(row=i+1, column=1)
            self.orbit_generator_optionmenus[v[0]] = temp
            
        Label(orbit_generator_frame, text="Semi-major axis length (m): ").grid(column=0)
        temp = Entry(orbit_generator_frame, textvariable=self.orbit_generator_inputs["semimajor_axis_length"])
        temp_var = self.orbit_generator_inputs["semimajor_axis_length"]
        temp.bind("<FocusIn>", lambda event:self.pause())
        temp.bind("<Return>", lambda event: temp_var.set(temp.get() if temp.get().replace(".", "").isnumeric() else "Numeric input only"))
        temp.grid(row=3, column=1)
        
        Button(orbit_generator_frame, text="Generate stable orbit", command=lambda:self.gen_orbit_from_inputs()).grid()
        
        Button(self.root, text="Add body", command=lambda:self.invert_adding_body()).grid()
        
    
    def manual_step(self):
        self.sim.step()
        self.refresh_selected_body_info()    
    
        
    def decenter_on_selected_body(self):
        if self.centered_body:
            b = self.centered_body
            self.centered_body = None
            self.x_offset = -b.x*self.body_scaling
            self.y_offset = b.y*self.body_scaling
            
        
    def center_on_selected_body(self):
        if self.selected_body:
            self.centered_body = self.selected_body
            self.x_offset, self.y_offset = 0, 0
        
        
    def delete_selected_body(self):
        if self.selected_body is None: return
        body_names = [b.name for b in self.sim.bodies]
        bi = body_names.index(self.selected_body.name)
        del self.sim.bodies[bi]
        self.sim.num_bodies -= 1
        self.selected_body = None
        self.refresh_selected_body_info()
        self.draw_bodies()
        
        
    def invert_adding_body(self):
        self.adding_body = not self.adding_body
        
        
    def gen_orbit_from_inputs(self):
        valid = True
        for i in self.orbit_generator_inputs:
            if i == "-" or i == "Numeric input only":
                valid = False
                
        if valid:
            body_names = [b.name for b in self.sim.bodies]
            orbiting_body_name = self.orbit_generator_inputs["orbiting_body"].get()
            body_to_orbit_around_name = self.orbit_generator_inputs["body_to_orbit_around"].get()
            semi_major_axis_length = float(self.orbit_generator_inputs["semimajor_axis_length"].get())
            b1 = self.sim.bodies[body_names.index(orbiting_body_name)]
            b2 = self.sim.bodies[body_names.index(body_to_orbit_around_name)]
            try:
                self.sim.generate_stable_orbit(b1, b2, semi_major_axis_length)
                for i in self.orbit_generator_inputs.values():
                    i.set("-")
            except ValueError:
                self.orbit_generator_inputs["semimajor_axis_length"].set("Too small!")

        
    def set_selected_body_field(self, field):
        entry_widget = self.body_info_entries[field]
        inputted_value = entry_widget.get()
        b = self.selected_body
        if field not in ("name", "is_immobile"): # The input must be numeric
            try:
                new_value = float(inputted_value)
                if field == "v" and (b.vx**2+b.vy**2) != 0:
                    k = math.sqrt(new_value**2/(b.vx**2+b.vy**2)) # Scale factor for the x and y velocities
                    exec(f"b.vx *= {k}")
                    exec(f"b.vy *= {k}")
                else:
                    exec(f"b.{field} = {new_value}")
            except ValueError: # Incorrect input
                pass
        elif field == "is_immobile":
            if inputted_value.lower() in ("true", "false"):
                exec(f"b.{field} = {inputted_value.lower().capitalize()}")
            else:
                pass
        else:
            if inputted_value not in [b.name for b in self.sim.bodies]:
                exec(f"b.name = '{inputted_value}'")
        self.refresh_selected_body_info()

       
    def set_fps(self, event):
        self.fps = int(event)
    
    
    def set_timestep(self, event):
        self.sim.t = float(event)
        
        
    def set_velocity_arrow_scale(self, event):
        self.velocity_arrow_scale = float(event)**2
        
        
    def set_acceleration_arrow_scale(self, event):
        self.acceleration_arrow_scale = float(event)**2
        
    
    def kill(self):
        self.root.destroy()
        pygame.quit()
    
    
    def pause(self):
        self.paused = True
    
    
    def un_pause_or_pause(self):
        self.paused = not self.paused
    

    def gui_mainloop(self):
        while True:
            self.window.fill("black") # Clear the window
            self.check_events()
            self.draw_grid()
            self.refresh_names_dropdown()
            self.draw_bodies()
            if not self.paused: 
                merge_ocurred = self.sim.step()
                if merge_ocurred or self.selected_body: self.refresh_selected_body_info()
                
            self.root.update()
            pygame.display.update()
            self.clock.tick(self.fps)
            
            
    def draw_grid(self):
        grid_x_offset = self.width//2-(self.width//(2*self.square_length))*self.square_length
        grid_y_offset = self.height//2-(self.height//(2*self.square_length))*self.square_length
        
        v = (-self.x_offset-self.width/2)/self.square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_x_offset = -int(v2*self.square_length)
        
        v = (-self.y_offset-self.height/2)/self.square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_y_offset = -int(v2*self.square_length)
        
        # Draw small grid squares
        for w in range(grid_x_offset, self.width, self.square_length):
            pygame.draw.line(self.window, "#191919", (w, 0), (w, self.height))
        for h in range(grid_y_offset, self.height, self.square_length):
            pygame.draw.line(self.window, "#191919", (0, h), (self.width, h))
            
        # Draw large grid squares, containing 5x5 small grid squares
        self.large_square_length = self.square_length*5
        grid_x_offset = self.width//2-(self.width//(2*self.large_square_length))*self.large_square_length
        grid_y_offset = self.height//2-(self.height//(2*self.large_square_length))*self.large_square_length
        
        v = (-self.x_offset-self.width/2)/self.large_square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_x_offset = -int(v2*self.large_square_length)
        
        v = (-self.y_offset-self.height/2)/self.large_square_length
        v2 = abs(math.trunc(v))-abs(v)
        if v > 0: v2 *= -1
        grid_y_offset = -int(v2*self.large_square_length)
        
        for w in range(grid_x_offset, self.width, self.large_square_length):
            pygame.draw.line(self.window, "#444444", (w, 0), (w, self.height))
        for h in range(grid_y_offset, self.height, self.large_square_length):
            pygame.draw.line(self.window, "#444444", (0, h), (self.width, h))
            
        # Draw center lighter grid lines
        pygame.draw.line(self.window, "#BBBBBB", (self.width//2+self.x_offset, 0), 
                        (self.width//2+self.x_offset, self.height))
        pygame.draw.line(self.window, "#BBBBBB", (0, self.height//2+self.y_offset), 
                        (self.width, self.height//2+self.y_offset))
            

    def refresh_selected_body_info(self):
        cx, cy = 0, 0
        if self.centered_body:
            cx = self.centered_body.x
            cy = self.centered_body.y
        
        b = self.selected_body
        if b is None:
            for i in self.selected_body_info.keys():
                self.selected_body_info[i].set("-") 
        else:
            assignments = {"name": b.name, "m": f"{b.m:.3e}", "r": f"{b.r:.3e}", "x": round(b.x-cx), "y": round(b.y-cy),
                           "is_immobile": b.is_immobile, "v": f"{b.v:.3e}", "vx": f"{b.vx:.3e}", "vy": f"{b.vy:.3e}", 
                           "a": f"{b.a:.3e}", "ax": f"{b.ax:.3e}", "ay": f"{b.ay:.3e}"}
            
            for i in self.selected_body_info.keys():
                self.selected_body_info[i].set(str(assignments[i]))
            
            
    def refresh_names_dropdown(self):
        menu1 = self.orbit_generator_optionmenus["orbiting_body"]["menu"]
        menu1_var = self.orbit_generator_inputs["orbiting_body"]
        menu1.delete(0, menu1.index(END))
        [menu1.add_command(label=b.name, command=lambda name=b.name: menu1_var.set(name)) for b in self.sim.bodies]
        
        menu2 = self.orbit_generator_optionmenus["body_to_orbit_around"]["menu"]
        menu2_var = self.orbit_generator_inputs["body_to_orbit_around"]
        menu2.delete(0, menu2.index(END))
        for b in self.sim.bodies:
            if b.name != self.orbit_generator_inputs["orbiting_body"].get():
                menu2.add_command(label=b.name, command=lambda name=b.name: menu2_var.set(name))
        
        if menu2_var.get() == menu1_var.get():
            menu2_var.set("-")
                        
            
    def check_events(self):
        for event in pygame.event.get():
            if event.type == pygame.MOUSEWHEEL:
                self.body_scaling *= 1+event.y*0.1 # Zoom is increased or decreased by 10% on zoom
                self.x_offset *= 1+event.y*0.1
                self.y_offset *= 1+event.y*0.1                
                
                self.square_length = int(self.square_length*(1+event.y*0.1))
                if self.square_length < 10: self.square_length = 30
                elif self.square_length > 30: self.square_length = 10
            elif event.type == pygame.MOUSEBUTTONDOWN and event.button not in (4, 5):
                centering_offset_x = 0
                centering_offset_y = 0
                if self.centered_body:
                    centering_offset_x = self.centered_body.x*self.body_scaling
                    centering_offset_y = self.centered_body.y*self.body_scaling
                    
                screen_x, screen_y = event.pos
                # Convert screen pixel coordinates to simulation coordinates
                sim_x = (screen_x-self.width//2-self.x_offset+centering_offset_x)/self.body_scaling
                sim_y = (-screen_y+self.height//2+self.y_offset+centering_offset_y)/self.body_scaling

                if self.adding_body and screen_x > 275:
                    self.pause()
                    self.sim.add_body(10**10, 5, sim_x, sim_y, f"Body {self.sim.num_bodies+1}", False)
                    self.selected_body = self.sim.bodies[-1]
                else:  
                    self.mouse_held = True
                    # Check all bodies to see if one was clicked on
                    for b in self.sim.bodies:
                        # Check if the distance between the mouse click and the center of the body is <= radius
                        if math.dist((b.x, b.y), (sim_x, sim_y)) <= b.r:
                            if self.selected_body == b: self.selected_body = None
                            else: self.selected_body = b
                            break
                
                self.refresh_selected_body_info()
                    
            elif event.type == pygame.MOUSEBUTTONUP:
                self.mouse_held = False
            elif event.type == pygame.MOUSEMOTION and self.mouse_held:
                self.x_offset += event.rel[0]
                self.y_offset += event.rel[1]
                
                
    def draw_bodies(self):
        if self.adding_body:
            mouse_pos = pygame.mouse.get_pos()
            pygame.draw.circle(self.window, "blue", mouse_pos, 5)
            
        centering_offset_x = 0
        centering_offset_y = 0
        if self.centered_body:
            centering_offset_x = self.centered_body.x*self.body_scaling
            centering_offset_y = self.centered_body.y*self.body_scaling
        
        for b in self.sim.bodies:
            x = b.x*self.body_scaling+self.width//2+self.x_offset-centering_offset_x
            y = -b.y*self.body_scaling+self.height//2+self.y_offset+centering_offset_y
            color = "yellow" if b == self.selected_body else "blue"
            pygame.draw.circle(self.window, color, (x, y), math.ceil(b.r*self.body_scaling))
            if b == self.selected_body and not b.is_immobile: # Draw the vector arrow showing its direction of motion
                
                x1, y1 = x+b.vx*self.velocity_arrow_scale*self.body_scaling, y-b.vy*self.velocity_arrow_scale*self.body_scaling
                pygame.draw.line(self.window, "red", (x, y), (x1, y1), width=math.ceil(b.r*self.body_scaling*0.5+5))
                mag = max(math.sqrt(b.vx**2+b.vy**2), 0.000001)
                length = max(b.r*self.body_scaling, 10)
                k = math.sqrt(3)*length/mag
                pygame.draw.polygon(self.window, "red", ((x1+length/mag*b.vy, y1-length/mag*-b.vx),
                                                         (x1+length/mag*-b.vy, y1-length/mag*b.vx),
                                                         (x1+b.vx*k, y1-b.vy*k)))
                
                x2, y2 = x+b.ax*self.acceleration_arrow_scale*self.body_scaling, y-b.ay*self.acceleration_arrow_scale*self.body_scaling
                pygame.draw.line(self.window, "purple", (x, y), (x2, y2), width=math.ceil(b.r*self.body_scaling*0.5+5))
                mag = max(math.sqrt(b.ax**2+b.ay**2), 0.000001)
                length = max(b.r*self.body_scaling, 10)
                k = math.sqrt(3)*length/mag
                pygame.draw.polygon(self.window, "purple", ((x2+length/mag*b.ay, y2-length/mag*-b.ax),
                                                         (x2+length/mag*-b.ay, y2-length/mag*b.ax),
                                                         (x2+b.ax*k, y2-b.ay*k)))

            # Draw the ellipse for an orbit, if it is being made by the user
            body_names = [b.name for b in self.sim.bodies]
            orbiting_body_name = self.orbit_generator_inputs["orbiting_body"].get()
            body_to_orbit_around_name = self.orbit_generator_inputs["body_to_orbit_around"].get()
            semi_major_axis_length = self.orbit_generator_inputs["semimajor_axis_length"].get()
            if orbiting_body_name == b.name and body_to_orbit_around_name in body_names and semi_major_axis_length.replace(".", "").isnumeric():
                b1 = self.sim.bodies[body_names.index(orbiting_body_name)]
                b2 = self.sim.bodies[body_names.index(body_to_orbit_around_name)]
                semi_major_axis_length = a = float(semi_major_axis_length)
                
                # Calculate the semi-minor axis length
                θ = math.atan2(b2.y-b1.y, b2.x-b1.x) # Angle from orbiting body to the body it orbits around
                apoapsis = ra = math.dist((b1.x, b1.y), (b2.x, b2.y))
                periapsis = rp = math.dist((b1.x+math.cos(θ)*2*a, b1.y+math.sin(θ)*2*a), (b2.x, b2.y))
                orbit_eccentricity = (ra-rp)/(ra+rp)             
                semi_minor_axis_length = b = a*math.sqrt(1-orbit_eccentricity**2)
                
                # Draw and rotate the ellipse
                ecx, ecy = ellipse_center = (self.width//2+self.body_scaling*(b1.x+a*math.cos(θ)), 
                                             self.height//2-self.body_scaling*(b1.y+a*math.sin(θ)))
                ellipse_surface = pygame.Surface((2*a*self.body_scaling, 2*b*self.body_scaling), pygame.SRCALPHA)
                pygame.draw.ellipse(ellipse_surface, "green", (0, 0, 2*a*self.body_scaling, 2*b*self.body_scaling), 5)
                ellipse_surface = pygame.transform.rotate(ellipse_surface, math.degrees(θ))
                new_rect = ellipse_surface.get_rect()
                w, h = new_rect.size
                self.window.blit(ellipse_surface, (ecx-w/2+self.x_offset, ecy-h/2+self.y_offset))


In [101]:
# Add acceleration property to bodies
# Add acceleration arrow to bodies
# Experiment with optimization (e.g. acceleration towards center of mass and grouping, "Regularisation of close encounters", "Ahmad-Cohen neighbour scheme")
# Add Barnes-Hut optimization
# Research lagrange points and try to show them
# Research equipotential lines and try to show them
# 4. Illustrate gravitational field around the body
# 7. Keep logs of velocity/other parameters and then graph them on-screen
# 8. Make it more user friendly
# 11. Optimize
# 12. More features! Ask teachers.

In [102]:
GUI()

error: video system not initialized